In [ ]:
#Prerequisites
#!pip install torch torchvision
#!pip install torch-geometric
#!pip install gudhi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.3 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_networkx
import networkx as nx
import numpy as np
import gudhi as gd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
import pickle
import os

In [ ]:
# Load PROTEINS data
def load_proteins_dataset():
    dataset = TUDataset(root='data/TUDataset', name='PROTEINS')
    graphs = []
    labels = []
    for data in dataset:
        G = to_networkx(data, to_undirected=True)
        graphs.append(G)
        labels.append(data.y.item())
    return graphs, labels

graphs_proteins, labels_proteins = load_proteins_dataset()

# Can reduce the size of the data for the tests :
# subset_size = 100
# graphs_proteins = graphs_proteins[:subset_size]
# labels_proteins = labels_proteins[:subset_size]

In [ ]:
# Second approach : simulate data and test the 2 methods
def generate_simulated_data(num_graphs_per_class=100, num_nodes=30):
    graphs = []
    labels = []

    for _ in range(num_graphs_per_class):
        G = nx.erdos_renyi_graph(n=num_nodes, p=0.05)
        if not nx.is_connected(G):
            largest_cc = max(nx.connected_components(G), key=len)
            G = G.subgraph(largest_cc).copy()
        graphs.append(G)
        labels.append(0)

    for _ in range(num_graphs_per_class):
        G = nx.cycle_graph(n=num_nodes)
        extra_edges = int(0.1 * num_nodes)
        for _ in range(extra_edges):
            u = np.random.randint(0, num_nodes)
            v = np.random.randint(0, num_nodes)
            if u != v:
                G.add_edge(u, v)
        graphs.append(G)
        labels.append(1)

    return graphs, labels

graphs_simulated, labels_simulated = generate_simulated_data(num_graphs_per_class=100, num_nodes=30)

In [ ]:
# Main functions

"Function to convert graphs into distance matrices"
def graph_to_distance_matrix(G):
    length = dict(nx.all_pairs_shortest_path_length(G))
    nodes = list(G.nodes())
    n = len(nodes)
    distance_matrix = np.zeros((n, n))
    for i, u in enumerate(nodes):
        for j, v in enumerate(nodes):
            if v in length[u]:
                distance_matrix[i, j] = length[u][v]
            else:
                distance_matrix[i, j] = np.inf
    return distance_matrix

"Function to compute persistence diagrams"
def compute_persistence_diagram(distance_matrix):
    rips_complex = gd.RipsComplex(distance_matrix=distance_matrix)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
    simplex_tree.compute_persistence()
    diag = simplex_tree.persistence_intervals_in_dimension(1)
    if len(diag) == 0:
        diag = np.array([[0.0, 0.0]])
    return diag

def compute_persistence_diagrams_parallel(distance_matrices):
    results = Parallel(n_jobs=-1)(
        delayed(compute_persistence_diagram)(dm) for dm in distance_matrices
    )
    return results


In [ ]:
# Comptute persistence diagrams (PROTEINS dataset)
distance_matrices_proteins = [graph_to_distance_matrix(G) for G in graphs_proteins]

diagrams_path_proteins = 'persistence_diagrams_proteins.pkl'
if os.path.exists(diagrams_path_proteins):
    with open(diagrams_path_proteins, 'rb') as f:
        persistence_diagrams_proteins = pickle.load(f)
else:
    persistence_diagrams_proteins = compute_persistence_diagrams_parallel(distance_matrices_proteins)
    with open(diagrams_path_proteins, 'wb') as f:
        pickle.dump(persistence_diagrams_proteins, f)

In [ ]:
# Compute persistence diagrams (simulated dataset)
distance_matrices_simulated = [graph_to_distance_matrix(G) for G in graphs_simulated]
persistence_diagrams_simulated = compute_persistence_diagrams_parallel(distance_matrices_simulated)

In [ ]:
# Data processing
X_proteins = persistence_diagrams_proteins
y_proteins = np.array(labels_proteins)

# Labels
unique_labels_proteins = np.unique(y_proteins)
label_mapping_proteins = {label: idx for idx, label in enumerate(unique_labels_proteins)}
y_proteins = np.array([label_mapping_proteins[label] for label in y_proteins])

In [ ]:
# Simulated data
X_simulated = persistence_diagrams_simulated
y_simulated = np.array(labels_simulated)

In [ ]:
"Function to compute MTF features"
def compute_mtf_features(diagrams, num_features=13):
    features = []
    for diag in diagrams:
        persistences = diag[:, 1] - diag[:, 0]
        sorted_persistences = np.sort(persistences)[::-1]
        if len(sorted_persistences) < num_features:
            sorted_persistences = np.pad(
                sorted_persistences, (0, num_features - len(sorted_persistences)), 'constant'
            )
        else:
            sorted_persistences = sorted_persistences[:num_features]
        features.append(sorted_persistences)
    return np.array(features)

**Protein data**

MTF-SVM method


In [ ]:
mtf_features_proteins = compute_mtf_features(X_proteins)

# Scale features
scaler_mtf_proteins = StandardScaler()
mtf_features_proteins_scaled = scaler_mtf_proteins.fit_transform(mtf_features_proteins)

C_values = [0.1, 1, 10, 100]
best_score_mtf_proteins = 0
best_params_mtf_proteins = {}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Evaluation of MTF-SVM method (protein data)")
for C in C_values:
    accuracies = []
    for train_index, test_index in cv.split(mtf_features_proteins_scaled, y_proteins):
        X_train_cv, X_test_cv = mtf_features_proteins_scaled[train_index], mtf_features_proteins_scaled[test_index]
        y_train_cv, y_test_cv = y_proteins[train_index], y_proteins[test_index]
        clf = SVC(kernel='rbf', C=C)
        clf.fit(X_train_cv, y_train_cv)
        y_pred_cv = clf.predict(X_test_cv)
        acc = accuracy_score(y_test_cv, y_pred_cv)
        accuracies.append(acc)
    mean_accuracy = np.mean(accuracies)
    print(f"MTF-SVM (PROTEINS) - C: {C}, Mean accuracy: {mean_accuracy:.4f}")
    if mean_accuracy > best_score_mtf_proteins:
        best_score_mtf_proteins = mean_accuracy
        best_params_mtf_proteins = {'C': C}

print(f"\nBest parameters: {best_params_mtf_proteins}, Mean accuracy: {best_score_mtf_proteins:.4f}\n")

Evaluation of MTF-SVM method (protein data)
MTF-SVM (PROTEINS) - C: 0.1, Mean accuracy: 0.5957
MTF-SVM (PROTEINS) - C: 1, Mean accuracy: 0.5795
MTF-SVM (PROTEINS) - C: 10, Mean accuracy: 0.6047
MTF-SVM (PROTEINS) - C: 100, Mean accuracy: 0.5984

Best parameters: {'C': 10}, Mean accuracy: 0.6047



PWGK with RFF method

In [ ]:
# Hyperparameters
sigma_values = [0.1, 1.0, 10.0]
p_values = [1, 2, 5]
D_values = [500, 1000, 5000]
C_values = [0.1, 1, 10]

best_score_rff_proteins = 0
best_params_rff_proteins = {}

print("Evaluation of PGWK with RFF (protein data)")
for sigma in sigma_values:
    for p in p_values:
        for D in D_values:
            np.random.seed(42)
            omega = np.random.normal(0, 1 / sigma, size=(D, 2))
            b = np.random.uniform(0, 2 * np.pi, size=D)
            def rff_transform(diagrams):
                transformed_diagrams = []
                for diag in diagrams:
                    if len(diag) == 0:
                        transformed_diagrams.append(np.zeros(D))
                        continue
                    weights = np.abs(diag[:, 1] - diag[:, 0]) ** p
                    z = np.sqrt(2 / D) * np.cos(diag @ omega.T + b)
                    z_weighted = z * weights[:, np.newaxis]
                    z_agg = np.sum(z_weighted, axis=0)
                    transformed_diagrams.append(z_agg)
                return np.array(transformed_diagrams)
            X_transformed = rff_transform(X_proteins)
            scaler_rff = StandardScaler()
            X_transformed_scaled = scaler_rff.fit_transform(X_transformed)
            for C in C_values:
                accuracies = []
                for train_index, test_index in cv.split(X_transformed_scaled, y_proteins):
                    X_train_cv, X_test_cv = X_transformed_scaled[train_index], X_transformed_scaled[test_index]
                    y_train_cv, y_test_cv = y_proteins[train_index], y_proteins[test_index]
                    clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1/(C * len(y_train_cv)),
                                        max_iter=1000, tol=1e-3, random_state=42)
                    clf.fit(X_train_cv, y_train_cv)
                    y_pred_cv = clf.predict(X_test_cv)
                    acc = accuracy_score(y_test_cv, y_pred_cv)
                    accuracies.append(acc)
                mean_accuracy = np.mean(accuracies)
                print(f"PWGK-RFF (PROTEINS) - Sigma: {sigma}, p: {p}, D: {D}, C: {C}, Mean accuracy: {mean_accuracy:.4f}")
                if mean_accuracy > best_score_rff_proteins:
                    best_score_rff_proteins = mean_accuracy
                    best_params_rff_proteins = {'sigma': sigma, 'p': p, 'D': D, 'C': C}

print(f"\nBest parameters: {best_params_rff_proteins}, Mean accuracy: {best_score_rff_proteins:.4f}\n")


Evaluation of PGWK with RFF (protein data)
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 1, D: 500, C: 0.1, Mean accuracy: 0.5535
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 1, D: 500, C: 1, Mean accuracy: 0.5723
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 1, D: 500, C: 10, Mean accuracy: 0.5787
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 1, D: 1000, C: 0.1, Mean accuracy: 0.5598
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 1, D: 1000, C: 1, Mean accuracy: 0.5463
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 1, D: 1000, C: 10, Mean accuracy: 0.5777
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 1, D: 5000, C: 0.1, Mean accuracy: 0.5445
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 1, D: 5000, C: 1, Mean accuracy: 0.5526
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 1, D: 5000, C: 10, Mean accuracy: 0.5553
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 2, D: 500, C: 0.1, Mean accuracy: 0.5849
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 2, D: 500, C: 1, Mean accuracy: 0.5534
PWGK-RFF (PROTEINS) - Sigma: 0.1, p: 2, D: 500, C: 10, Mean accuracy: 0.5599
PWGK-RFF (PROTEINS) - Sigma

**Simulated data**

MTF-SVM method


In [ ]:
# MTF-SVM method with simulated data
mtf_features_simulated = compute_mtf_features(X_simulated)

scaler_mtf_simulated = StandardScaler()
mtf_features_simulated_scaled = scaler_mtf_simulated.fit_transform(mtf_features_simulated)

best_score_mtf_simulated = 0
best_params_mtf_simulated = {}

print("Evaluation of the MTF-SVM method (simulated data)")
for C in C_values:
    accuracies = []
    for train_index, test_index in cv.split(mtf_features_simulated_scaled, y_simulated):
        X_train_cv, X_test_cv = mtf_features_simulated_scaled[train_index], mtf_features_simulated_scaled[test_index]
        y_train_cv, y_test_cv = y_simulated[train_index], y_simulated[test_index]
        clf = SVC(kernel='rbf', C=C)
        clf.fit(X_train_cv, y_train_cv)
        y_pred_cv = clf.predict(X_test_cv)
        acc = accuracy_score(y_test_cv, y_pred_cv)
        accuracies.append(acc)
    mean_accuracy = np.mean(accuracies)
    print(f"MTF-SVM (simulated) - C: {C}, Mean accuracy: {mean_accuracy:.4f}")
    if mean_accuracy > best_score_mtf_simulated:
        best_score_mtf_simulated = mean_accuracy
        best_params_mtf_simulated = {'C': C}

print(f"\nBest parameters: {best_params_mtf_simulated}, Mean accuracy: {best_score_mtf_simulated:.4f}\n")

Evaluation of the MTF-SVM method (simulated data)
MTF-SVM (simulated) - C: 0.1, Mean accuracy: 0.9650
MTF-SVM (simulated) - C: 1, Mean accuracy: 0.9800
MTF-SVM (simulated) - C: 10, Mean accuracy: 0.9800

Best parameters: {'C': 1}, Mean accuracy: 0.9800



PWGK with RFF method

In [ ]:
best_score_rff_simulated = 0
best_params_rff_simulated = {}

print("Evaluation of PGWK with RFF (simulated data)")
for sigma in sigma_values:
    for p in p_values:
        for D in D_values:
            np.random.seed(42)
            omega = np.random.normal(0, 1 / sigma, size=(D, 2))
            b = np.random.uniform(0, 2 * np.pi, size=D)
            def rff_transform(diagrams):
                transformed_diagrams = []
                for diag in diagrams:
                    if len(diag) == 0:
                        transformed_diagrams.append(np.zeros(D))
                        continue
                    weights = np.abs(diag[:, 1] - diag[:, 0]) ** p
                    z = np.sqrt(2 / D) * np.cos(diag @ omega.T + b)
                    z_weighted = z * weights[:, np.newaxis]
                    z_agg = np.sum(z_weighted, axis=0)
                    transformed_diagrams.append(z_agg)
                return np.array(transformed_diagrams)
            X_transformed = rff_transform(X_simulated)
            scaler_rff = StandardScaler()
            X_transformed_scaled = scaler_rff.fit_transform(X_transformed)
            for C in C_values:
                accuracies = []
                for train_index, test_index in cv.split(X_transformed_scaled, y_simulated):
                    X_train_cv, X_test_cv = X_transformed_scaled[train_index], X_transformed_scaled[test_index]
                    y_train_cv, y_test_cv = y_simulated[train_index], y_simulated[test_index]
                    clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1/(C * len(y_train_cv)),
                                        max_iter=1000, tol=1e-3, random_state=42)
                    clf.fit(X_train_cv, y_train_cv)
                    y_pred_cv = clf.predict(X_test_cv)
                    acc = accuracy_score(y_test_cv, y_pred_cv)
                    accuracies.append(acc)
                mean_accuracy = np.mean(accuracies)
                print(f"PWGK-RFF (simulated) - Sigma: {sigma}, p: {p}, D: {D}, C: {C}, Mean accuracy: {mean_accuracy:.4f}")
                if mean_accuracy > best_score_rff_simulated:
                    best_score_rff_simulated = mean_accuracy
                    best_params_rff_simulated = {'sigma': sigma, 'p': p, 'D': D, 'C': C}

print(f"\nBest parameters: {best_params_rff_simulated}, Mean accuracy: {best_score_rff_simulated:.4f}\n")

Evaluation of PGWK with RFF (simulated data)
PWGK-RFF (simulated) - Sigma: 0.1, p: 1, D: 500, C: 0.1, Mean accuracy: 0.9550
PWGK-RFF (simulated) - Sigma: 0.1, p: 1, D: 500, C: 1, Mean accuracy: 0.9850
PWGK-RFF (simulated) - Sigma: 0.1, p: 1, D: 500, C: 10, Mean accuracy: 0.9850
PWGK-RFF (simulated) - Sigma: 0.1, p: 1, D: 1000, C: 0.1, Mean accuracy: 0.9600
PWGK-RFF (simulated) - Sigma: 0.1, p: 1, D: 1000, C: 1, Mean accuracy: 0.9850
PWGK-RFF (simulated) - Sigma: 0.1, p: 1, D: 1000, C: 10, Mean accuracy: 0.9850
PWGK-RFF (simulated) - Sigma: 0.1, p: 1, D: 5000, C: 0.1, Mean accuracy: 0.9750
PWGK-RFF (simulated) - Sigma: 0.1, p: 1, D: 5000, C: 1, Mean accuracy: 0.9850
PWGK-RFF (simulated) - Sigma: 0.1, p: 1, D: 5000, C: 10, Mean accuracy: 0.9850
PWGK-RFF (simulated) - Sigma: 0.1, p: 2, D: 500, C: 0.1, Mean accuracy: 0.9850
PWGK-RFF (simulated) - Sigma: 0.1, p: 2, D: 500, C: 1, Mean accuracy: 0.9850
PWGK-RFF (simulated) - Sigma: 0.1, p: 2, D: 500, C: 10, Mean accuracy: 0.9850
PWGK-RFF (sim

**Final results**

In [ ]:
# Print final results
print("Final results on PROTEINS dataset:")
print(f"Mean accuracy MTF-SVM method: {best_score_mtf_proteins:.4f}")
print(f"Mean accuracy PWGK with RFF method: {best_score_rff_proteins:.4f}\n")

print("Final results on simulated dataset:")
print(f"Mean accuracy MTF-SVM method: {best_score_mtf_simulated:.4f}")
print(f"Mean accuracy PWGK with RFF method: {best_score_rff_simulated:.4f}")

Final results on PROTEINS dataset:
Mean accuracy MTF-SVM method: 0.6047
Mean accuracy PWGK with RFF method: 0.6056

Final results on simulated data:
Mean accuracy MTF-SVM method: 0.9800
Mean accuracy PWGK with RFF method: 1.0000
